#  Projekt zur Datenanalyse_Classification of Wisconsin Breast Cancer

### Table of contents

### chapter 1: Data Preparation

1. Ordinal Encoding- Target variable
       
2. Feature Selecktion-SelectKBest
    1. pearon correlation  
    2. f_classif 
    3. Visualization of Feature selektion 
  
3. Feature Scaling   
   1. Standardization
   2. Normalization
    
### chapter 2: Neural Network

1. Split the dataset
    1. Cross Validation- to overcome overfitting
     
2. Building a model
    1. Input Layers- Hidden layers- Output layers
    2. Compile the model
       1. Optimization Moethod:
           1.Stochastic Gradient Descent
           2. Adam
           3. Learning rate
           
3. Fitting the model
4. Loss- Accuracy
5. Evaluation metrics
    1. Confusion_metrics
    
6. Classification threshold
    1. Adjsting Threshold
    2. ROC und AUC
    3. Visualization of ROC- AUC
7. Interpretation des modelles
    1. Shap analysis
 
### Chapter 3 : Support vector machine
1. Data recuperation
2. Data Visualization
3. Correlation of features
4. Data Selection
5. Fiting the model
6. testing the model
7. tunning the hyperparameters
8. Confusion matrix

###  Chapter 4: k-nearest neighbors

    



 
### **For more Information :** :[ https://www.downloadmaghaleh.com/wp-content/uploads/edd/maghaleh/1398/yosefi.wisconsin-min.pdf ]

### Dataset: Description
<a id="infos"></a>
* id
* diagnosis
* radius_mean
* texture_mean
* perimeter_mean
* area_mean
* smoothness_mean
* compactness_mean
* concavity_mean
* concave points_mean
* symmetry_mean
* fractal_dimension_mean
* radius_se
* texture_se
* perimeter_se
* area_se
* smoothness_se
* compactness_se
* concavity_se
* concave points_se
* symmetry_se ,
* fractal_dimension_se
* radius_worst
* texture_worst
* perimeter_worst
* area_worst
* smoothness_worst
* compactness_worst
* concavity_worst
* concave points_worst
* symmetry_worst
* fractal_dimension_worst
* Unnamed: 32

* This breast cancer databases was obtained from the University of Wisconsin Hospitals, Madison from Dr. William H. Wolberg.

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.impute import SimpleImputer

* Data characteristics
    * 569 Rows - 33 columns
   